In [2]:
PROJECT = !gcloud config get-value project

In [3]:
PROJECT=PROJECT[0]
BUCKET = PROJECT + '-dsongcp'

In [4]:
import os
os.environ['BUCKET'] = PROJECT + '-dsongcp'

In [5]:
from pyspark.sql import SparkSession

In [6]:
from pyspark import SparkContext

In [7]:
sc = SparkContext('local', 'logistic')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/17 15:12:44 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/11/17 15:12:44 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/11/17 15:12:44 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/11/17 15:12:44 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [8]:
spark = SparkSession \
    .builder \
    .appName('Logistic regression w/ Spark ML') \
    .getOrCreate()

In [9]:
from pyspark import SparkContext
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

In [10]:
# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

In [11]:
data = sc.textFile('gs://{}/spark/sample_svm_data.txt'.format(BUCKET))


In [12]:
parsedData = data.map(parsePoint)

In [13]:
# Build the model
model = LogisticRegressionWithLBFGS.train(parsedData)

22/11/17 15:12:56 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
22/11/17 15:13:04 WARN org.apache.spark.ml.util.Instrumentation: [0df419c1] Initial coefficients will be ignored! Its dimensions (1, 16) did not match the expected size (1, 16)
22/11/17 15:13:05 WARN com.github.fommil.netlib.BLAS: Failed to loa

In [14]:
# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))

In [15]:
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.36645962732919257
